In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
from place_groups import place_groups_middle_of_roads, place_groups_junctions

from sklearn.linear_model import LinearRegression
from datetime import timedelta
from keras.callbacks import ModelCheckpoint
import numpy as np
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import pandas as pd
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.recurrent import LSTM
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers import BatchNormalization
from keras.models import Sequential
from keras import regularizers

from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (16, 10)

# prevent tensorflow from allocating the entire GPU memory at once
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

Using TensorFlow backend.


In [14]:
DEFAULT_FLOW_FOR_MISSING_VALUES_AT_BEGINNING_OF_TIME_SERIES = 0
ONE_DAY_LAGS = 288
DATA_PATH = '/mnt/sdc1/inon/norrecampus/data/by_place_5min'
EXAMPLE_PLACE_ID = 'ChIJZaR1M1hSUkYRxP0WkwYYy_k'

THRESHOLD_HIGH_OUTLIER = 110
LOSS = 'mse'
MINI_BATCH_SIZE = 512
NUM_EPOCHS = 100
VALIDATION_SPLIT = 0.2
NUM_LAGS = 12
LSTM_STATE_SIZE = NUM_LAGS
NUM_OUTS_MEAN_SPEED_REGRESSION = 1
SPLIT_DATE = '2015-06-01'

In [4]:
def split_timestamp(ser):
    return pd\
        .DataFrame(ser)\
        .assign(day_of_week=lambda df: df.index.dayofweek, time_of_day=lambda df: df.index.time)\
        .reset_index()

In [5]:
def compute_error_statistics(errors_df, place_id):
    abs_errors = errors_df.error.abs()
    abs_errors_normalized = abs_errors / errors_df.flow_decile_true
    return pd.DataFrame({
        'corr': np.corrcoef(errors_df.flow_decile_predicted, errors_df.flow_decile_true)[0, 1],
        'mae': np.mean(abs_errors),
        'mape': np.mean(abs_errors_normalized),
        'mse': np.mean(abs_errors ** 2),
        'msne': np.mean(abs_errors_normalized ** 2),
        'rae': np.sum(abs_errors) / np.sum(np.abs(errors_df.flow_decile_true - np.mean(errors_df.flow_decile_true))),
        'rmse': np.sqrt(np.mean(abs_errors ** 2)),
        'rmsne': np.sqrt(np.mean(abs_errors_normalized ** 2)),
        'r2': max(0, 1 - np.sum(abs_errors ** 2) / np.sum((errors_df.flow_decile_true - np.mean(errors_df.flow_decile_true)) ** 2))
    }, index=[place_id])

In [18]:
class LearnerForFlowDecileOfOneSegmentWithoutDetrending(object):
    def __init__(self,
                 loss,
                 place_id, 
                 mini_batch_size,
                 num_epochs,
                 validation_split,
                 num_lags,
                 lstm_state_size,
                 num_outs,
                 split_date):
        self.df = None
        self.model = None
        
        self.loss = loss
        self.place_id = place_id
        self.mini_batch_size = mini_batch_size
        self.num_epochs = num_epochs
        self.validation_split = validation_split
        self.num_lags = num_lags
        self.lstm_state_size = lstm_state_size
        self.num_outs = num_outs
        self.split_date = split_date
        self.checkpoint_basename = 'simple_lstm_flow_%s.best.hdf5' % self.place_id      
    
    def baseline_lr(self):
        X_train, Y_train, X_test, Y_test = self._get_train_and_test_inputs()
        trained_lr = LinearRegression(fit_intercept=False).fit(X_train.values, Y_train.values)
        lr_predictions = trained_lr.predict(X_test).round(0)
        errors_df = self._individual_errors_without_interpolated_values(lr_predictions)
        return compute_error_statistics(errors_df, self.place_id)
                    
    def _imputation_for_missing_values(self, ser):
        missing_timestamps = pd\
            .date_range(min(ser.index.date), max(ser.index.date) + timedelta(days=1), freq='5T')\
            .difference(ser.index)
        df_with_nans_where_missing = pd.DataFrame(ser)\
            .join(pd.DataFrame(index=missing_timestamps), how='outer')\
            .assign(original_value=lambda df: df.iloc[:, 0])
        df_interpolated = df_with_nans_where_missing\
            .iloc[:, 0]\
            .interpolate()\
            .fillna(DEFAULT_FLOW_FOR_MISSING_VALUES_AT_BEGINNING_OF_TIME_SERIES)\
            .round()\
            .astype(np.int64)\
            .to_frame()\
            .assign(original_value=df_with_nans_where_missing.original_value)\
            .assign(is_interpolated=lambda df: df.iloc[:, 0] != df.iloc[:, 1])
        assert all(df_interpolated.flow_decile.isin(range(10)))
        return df_interpolated
        
    def _get_flows(self):
        if self.df is None:
            self.df = pd.read_csv(
                os.path.join(DATA_PATH, self.place_id + '.csv'),
                parse_dates=['start_interval_s', 'end_interval_s']\
            )[lambda df: df.start_interval_s >= '2015-01-01']\
            .rename(columns={'start_interval_s': 't', 'flow_bucket': 'flow_decile'})\
            .set_index('t')\
            .flow_decile
        return self.df

    def _get_train_and_test_inputs(self):
        flows_interpolated = self._imputation_for_missing_values(self._get_flows()).flow_decile
        lags = pd.concat([flows_interpolated.shift(x) for x in range(self.num_lags + 1)], axis=1)[self.num_lags:]
        train = lags[lags.index < self.split_date]
        X_train = train.iloc[:, 1:]
        Y_train = train.iloc[:, 0]
        test = lags[lags.index >= self.split_date]
        X_test = test.iloc[:, 1:]
        Y_test = test.iloc[:, 0]
        return X_train, Y_train, X_test, Y_test

    def _build_model(self):
        # TODO: better weights initialization
        self.model = Sequential()
        self.model.add(LSTM(self.lstm_state_size, input_shape=(self.num_lags, 1), return_sequences=False))
        # model.add(Dropout(0.2))
        #model.add(Dense(units=1, kernel_regularizer=regularizers.l2(0.00001)))
        self.model.add(Dense(units=self.num_outs, activation="linear")) # TODO: also try relu, as ordinal categories are non-negative
        self.model.compile(loss=self.loss, optimizer="rmsprop")  # TODO: try adam optimizer too, although rmsprop is the default go-to for RNN
    
    def load_best_model_from_disk(self):
        self._build_model()
        self.model.load_weights(self.checkpoint_basename)
    
    def create_and_train_model(self):
        self._build_model()
        X_train, Y_train, _, _ = self._get_train_and_test_inputs()
        self.model.fit(
            np.expand_dims(np.array(X_train.values), 2),
            Y_train.values,
            batch_size=self.mini_batch_size,
            epochs=self.num_epochs,
            validation_split=self.validation_split,
            # checkpoint best model
            callbacks=[ModelCheckpoint(
                self.checkpoint_basename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')],
            verbose=2)
        self.model.load_weights(self.checkpoint_basename)
    
    def predict(self):
        _, _, X_test, Y_test = self._get_train_and_test_inputs()
        predictions_rounded = self.model.predict(np.expand_dims(X_test, 2)).flatten().round(0)
        errors_df = self._individual_errors_without_interpolated_values(predictions_rounded)
        return compute_error_statistics(errors_df, self.place_id)
        
    def _individual_errors_without_interpolated_values(self, predictions):
        _, _, _, Y_test = self._get_train_and_test_inputs()
        ser_predictions = pd.Series(predictions, index=Y_test.index, name='flow_decile')
        interpolated_timestamps = self\
            ._imputation_for_missing_values(self._get_flows())\
            [lambda df: df.is_interpolated]\
            .index
        return Y_test.to_frame()\
            .join(ser_predictions.to_frame(), lsuffix='_true', rsuffix='_predicted')\
            .loc[lambda df: df.index.difference(interpolated_timestamps)]\
            .assign(error=lambda df: df.flow_decile_true - df.flow_decile_predicted)

In [12]:
def one_place(place_id):
    learner = LearnerForFlowDecileOfOneSegmentWithoutDetrending(
        num_outs=1,
        loss=LOSS,
        place_id=place_id, 
        mini_batch_size=MINI_BATCH_SIZE,
        num_epochs=NUM_EPOCHS,
        validation_split=VALIDATION_SPLIT,
        num_lags=NUM_LAGS,
        lstm_state_size=LSTM_STATE_SIZE,
        split_date=SPLIT_DATE)
    if place_id == 'ChIJsVzlKx5TUkYRjzAT9WOeWcA':
        learner.create_and_train_model()
    else:
        learner.load_best_model_from_disk()
    return learner.predict()

def multiple_places(place_group, place_group_name):
    places = [lst[len(lst) // 2] for lst in place_group]
    return pd.concat(map(one_place, places))\
        .to_csv('%s_without_ordinal_regression.csv' % place_group_name)

In [8]:
multiple_places(place_groups_middle_of_roads, 'place_groups_middle_of_roads')

Train on 34780 samples, validate on 8696 samples
Epoch 1/100
Epoch 00001: val_loss improved from inf to 37.95083, saving model to simple_lstm_flow_ChIJ7feGfFdSUkYRmmsyva5Yh7g.best.hdf5
 - 2s - loss: 45.0627 - val_loss: 37.9508
Epoch 2/100
Epoch 00002: val_loss improved from 37.95083 to 22.68849, saving model to simple_lstm_flow_ChIJ7feGfFdSUkYRmmsyva5Yh7g.best.hdf5
 - 2s - loss: 28.5274 - val_loss: 22.6885
Epoch 3/100
Epoch 00003: val_loss improved from 22.68849 to 11.22042, saving model to simple_lstm_flow_ChIJ7feGfFdSUkYRmmsyva5Yh7g.best.hdf5
 - 2s - loss: 15.7465 - val_loss: 11.2204
Epoch 4/100
Epoch 00004: val_loss improved from 11.22042 to 4.75484, saving model to simple_lstm_flow_ChIJ7feGfFdSUkYRmmsyva5Yh7g.best.hdf5
 - 2s - loss: 7.1146 - val_loss: 4.7548
Epoch 5/100
Epoch 00005: val_loss improved from 4.75484 to 2.10492, saving model to simple_lstm_flow_ChIJ7feGfFdSUkYRmmsyva5Yh7g.best.hdf5
 - 1s - loss: 3.0666 - val_loss: 2.1049
Epoch 6/100
Epoch 00006: val_loss improved from 

Epoch 00067: val_loss did not improve
 - 1s - loss: 0.4002 - val_loss: 0.3441
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 2s - loss: 0.3963 - val_loss: 0.3756
Epoch 69/100
Epoch 00069: val_loss improved from 0.34178 to 0.33967, saving model to simple_lstm_flow_ChIJ7feGfFdSUkYRmmsyva5Yh7g.best.hdf5
 - 1s - loss: 0.3999 - val_loss: 0.3397
Epoch 70/100
Epoch 00070: val_loss did not improve
 - 2s - loss: 0.3949 - val_loss: 0.3428
Epoch 71/100
Epoch 00071: val_loss did not improve
 - 1s - loss: 0.3972 - val_loss: 0.3587
Epoch 72/100
Epoch 00072: val_loss did not improve
 - 1s - loss: 0.3939 - val_loss: 0.3404
Epoch 73/100
Epoch 00073: val_loss did not improve
 - 2s - loss: 0.3959 - val_loss: 0.3553
Epoch 74/100
Epoch 00074: val_loss did not improve
 - 1s - loss: 0.3945 - val_loss: 0.3716
Epoch 75/100
Epoch 00075: val_loss did not improve
 - 1s - loss: 0.3939 - val_loss: 0.3832
Epoch 76/100
Epoch 00076: val_loss did not improve
 - 2s - loss: 0.3969 - val_loss: 0.3561
Epoch 77/100
E

Epoch 36/100
Epoch 00036: val_loss improved from 0.40448 to 0.40133, saving model to simple_lstm_flow_ChIJ3WKEa1dSUkYR4s4cJpaCZfQ.best.hdf5
 - 2s - loss: 0.4782 - val_loss: 0.4013
Epoch 37/100
Epoch 00037: val_loss improved from 0.40133 to 0.40116, saving model to simple_lstm_flow_ChIJ3WKEa1dSUkYR4s4cJpaCZfQ.best.hdf5
 - 2s - loss: 0.4807 - val_loss: 0.4012
Epoch 38/100
Epoch 00038: val_loss did not improve
 - 2s - loss: 0.4771 - val_loss: 0.4192
Epoch 39/100
Epoch 00039: val_loss did not improve
 - 2s - loss: 0.4714 - val_loss: 0.4382
Epoch 40/100
Epoch 00040: val_loss did not improve
 - 2s - loss: 0.4705 - val_loss: 0.4472
Epoch 41/100
Epoch 00041: val_loss did not improve
 - 2s - loss: 0.4663 - val_loss: 0.4246
Epoch 42/100
Epoch 00042: val_loss did not improve
 - 2s - loss: 0.4644 - val_loss: 0.5192
Epoch 43/100
Epoch 00043: val_loss did not improve
 - 2s - loss: 0.4644 - val_loss: 0.4676
Epoch 44/100
Epoch 00044: val_loss did not improve
 - 2s - loss: 0.4602 - val_loss: 0.4045
Epo

Epoch 10/100
Epoch 00010: val_loss improved from 0.52195 to 0.52173, saving model to simple_lstm_flow_ChIJCy4tOFhSUkYRBiS5iJUrGnY.best.hdf5
 - 2s - loss: 0.5823 - val_loss: 0.5217
Epoch 11/100
Epoch 00011: val_loss did not improve
 - 2s - loss: 0.5700 - val_loss: 0.5923
Epoch 12/100
Epoch 00012: val_loss improved from 0.52173 to 0.46996, saving model to simple_lstm_flow_ChIJCy4tOFhSUkYRBiS5iJUrGnY.best.hdf5
 - 2s - loss: 0.5601 - val_loss: 0.4700
Epoch 13/100
Epoch 00013: val_loss improved from 0.46996 to 0.46729, saving model to simple_lstm_flow_ChIJCy4tOFhSUkYRBiS5iJUrGnY.best.hdf5
 - 2s - loss: 0.5468 - val_loss: 0.4673
Epoch 14/100
Epoch 00014: val_loss improved from 0.46729 to 0.46320, saving model to simple_lstm_flow_ChIJCy4tOFhSUkYRBiS5iJUrGnY.best.hdf5
 - 2s - loss: 0.5407 - val_loss: 0.4632
Epoch 15/100
Epoch 00015: val_loss improved from 0.46320 to 0.45284, saving model to simple_lstm_flow_ChIJCy4tOFhSUkYRBiS5iJUrGnY.best.hdf5
 - 2s - loss: 0.5310 - val_loss: 0.4528
Epoch 16/

Epoch 77/100
Epoch 00077: val_loss did not improve
 - 2s - loss: 0.3847 - val_loss: 0.3448
Epoch 78/100
Epoch 00078: val_loss did not improve
 - 2s - loss: 0.3849 - val_loss: 0.3420
Epoch 79/100
Epoch 00079: val_loss did not improve
 - 2s - loss: 0.3851 - val_loss: 0.3417
Epoch 80/100
Epoch 00080: val_loss did not improve
 - 1s - loss: 0.3855 - val_loss: 0.3638
Epoch 81/100
Epoch 00081: val_loss did not improve
 - 2s - loss: 0.3841 - val_loss: 0.3432
Epoch 82/100
Epoch 00082: val_loss improved from 0.34040 to 0.33763, saving model to simple_lstm_flow_ChIJCy4tOFhSUkYRBiS5iJUrGnY.best.hdf5
 - 1s - loss: 0.3798 - val_loss: 0.3376
Epoch 83/100
Epoch 00083: val_loss did not improve
 - 2s - loss: 0.3801 - val_loss: 0.3692
Epoch 84/100
Epoch 00084: val_loss did not improve
 - 1s - loss: 0.3833 - val_loss: 0.3419
Epoch 85/100
Epoch 00085: val_loss did not improve
 - 1s - loss: 0.3840 - val_loss: 0.3622
Epoch 86/100
Epoch 00086: val_loss did not improve
 - 1s - loss: 0.3818 - val_loss: 0.3442
E

...

Epoch 92/100
Epoch 00092: val_loss did not improve
 - 2s - loss: 0.2111 - val_loss: 0.2215
Epoch 93/100
Epoch 00093: val_loss did not improve
 - 1s - loss: 0.2137 - val_loss: 0.2047
Epoch 94/100
Epoch 00094: val_loss improved from 0.17694 to 0.17549, saving model to simple_lstm_flow_ChIJC9G6K_5SUkYRWa6ieQhstlo.best.hdf5
 - 2s - loss: 0.2120 - val_loss: 0.1755
Epoch 95/100
Epoch 00095: val_loss did not improve
 - 2s - loss: 0.2101 - val_loss: 0.2051
Epoch 96/100
Epoch 00096: val_loss did not improve
 - 2s - loss: 0.2129 - val_loss: 0.1764
Epoch 97/100
Epoch 00097: val_loss did not improve
 - 1s - loss: 0.2095 - val_loss: 0.1773
Epoch 98/100
Epoch 00098: val_loss did not improve
 - 2s - loss: 0.2105 - val_loss: 0.1885
Epoch 99/100
Epoch 00099: val_loss did not improve
 - 1s - loss: 0.2086 - val_loss: 0.1865
Epoch 100/100
Epoch 00100: val_loss did not improve
 - 2s - loss: 0.2077 - val_loss: 0.2031
Train on 34780 samples, validate on 8696 samples
Epoch 1/100
Epoch 00001: val_loss improved

Epoch 00053: val_loss did not improve
 - 1s - loss: 0.2106 - val_loss: 0.1887
Epoch 54/100
Epoch 00054: val_loss did not improve
 - 1s - loss: 0.2113 - val_loss: 0.1717
Epoch 55/100
Epoch 00055: val_loss did not improve
 - 2s - loss: 0.2097 - val_loss: 0.1802
Epoch 56/100
Epoch 00056: val_loss did not improve
 - 1s - loss: 0.2092 - val_loss: 0.1832
Epoch 57/100
Epoch 00057: val_loss did not improve
 - 1s - loss: 0.2105 - val_loss: 0.1899
Epoch 58/100
Epoch 00058: val_loss did not improve
 - 1s - loss: 0.2099 - val_loss: 0.2021
Epoch 59/100
Epoch 00059: val_loss did not improve
 - 1s - loss: 0.2091 - val_loss: 0.1758
Epoch 60/100
Epoch 00060: val_loss did not improve
 - 1s - loss: 0.2093 - val_loss: 0.1852
Epoch 61/100
Epoch 00061: val_loss did not improve
 - 1s - loss: 0.2090 - val_loss: 0.1745
Epoch 62/100
Epoch 00062: val_loss did not improve
 - 1s - loss: 0.2096 - val_loss: 0.1723
Epoch 63/100
Epoch 00063: val_loss did not improve
 - 1s - loss: 0.2083 - val_loss: 0.2049
Epoch 64/100

Epoch 00024: val_loss did not improve
 - 1s - loss: 0.3303 - val_loss: 0.2916
Epoch 25/100
Epoch 00025: val_loss did not improve
 - 1s - loss: 0.3295 - val_loss: 0.2876
Epoch 26/100
Epoch 00026: val_loss did not improve
 - 1s - loss: 0.3263 - val_loss: 0.2956
Epoch 27/100
Epoch 00027: val_loss did not improve
 - 1s - loss: 0.3256 - val_loss: 0.3248
Epoch 28/100
Epoch 00028: val_loss did not improve
 - 1s - loss: 0.3226 - val_loss: 0.2881
Epoch 29/100
Epoch 00029: val_loss did not improve
 - 1s - loss: 0.3228 - val_loss: 0.2886
Epoch 30/100
Epoch 00030: val_loss did not improve
 - 1s - loss: 0.3210 - val_loss: 0.2947
Epoch 31/100
Epoch 00031: val_loss did not improve
 - 1s - loss: 0.3208 - val_loss: 0.2836
Epoch 32/100
Epoch 00032: val_loss did not improve
 - 1s - loss: 0.3199 - val_loss: 0.3207
Epoch 33/100
Epoch 00033: val_loss improved from 0.28330 to 0.26840, saving model to simple_lstm_flow_ChIJsde_glVSUkYRlK8i5BD3BOA.best.hdf5
 - 1s - loss: 0.3182 - val_loss: 0.2684
Epoch 34/100
E

Epoch 3/100
Epoch 00003: val_loss improved from 17.39945 to 7.25301, saving model to simple_lstm_flow_ChIJozaGTFZSUkYRjY7TpiWcsro.best.hdf5
 - 1s - loss: 11.9538 - val_loss: 7.2530
Epoch 4/100
Epoch 00004: val_loss improved from 7.25301 to 2.19098, saving model to simple_lstm_flow_ChIJozaGTFZSUkYRjY7TpiWcsro.best.hdf5
 - 1s - loss: 3.8369 - val_loss: 2.1910
Epoch 5/100
Epoch 00005: val_loss improved from 2.19098 to 1.11386, saving model to simple_lstm_flow_ChIJozaGTFZSUkYRjY7TpiWcsro.best.hdf5
 - 1s - loss: 1.5010 - val_loss: 1.1139
Epoch 6/100
Epoch 00006: val_loss improved from 1.11386 to 0.70921, saving model to simple_lstm_flow_ChIJozaGTFZSUkYRjY7TpiWcsro.best.hdf5
 - 1s - loss: 0.8101 - val_loss: 0.7092
Epoch 7/100
Epoch 00007: val_loss improved from 0.70921 to 0.61402, saving model to simple_lstm_flow_ChIJozaGTFZSUkYRjY7TpiWcsro.best.hdf5
 - 1s - loss: 0.6138 - val_loss: 0.6140
Epoch 8/100
Epoch 00008: val_loss improved from 0.61402 to 0.53415, saving model to simple_lstm_flow_Ch

Epoch 69/100
Epoch 00069: val_loss did not improve
 - 2s - loss: 0.3448 - val_loss: 0.3393
Epoch 70/100
Epoch 00070: val_loss improved from 0.32285 to 0.31921, saving model to simple_lstm_flow_ChIJozaGTFZSUkYRjY7TpiWcsro.best.hdf5
 - 1s - loss: 0.3434 - val_loss: 0.3192
Epoch 71/100
Epoch 00071: val_loss did not improve
 - 2s - loss: 0.3463 - val_loss: 0.3540
Epoch 72/100
Epoch 00072: val_loss did not improve
 - 1s - loss: 0.3441 - val_loss: 0.3353
Epoch 73/100
Epoch 00073: val_loss did not improve
 - 1s - loss: 0.3445 - val_loss: 0.4123
Epoch 74/100
Epoch 00074: val_loss did not improve
 - 2s - loss: 0.3442 - val_loss: 0.3286
Epoch 75/100
Epoch 00075: val_loss did not improve
 - 1s - loss: 0.3426 - val_loss: 0.3428
Epoch 76/100
Epoch 00076: val_loss did not improve
 - 1s - loss: 0.3446 - val_loss: 0.3642
Epoch 77/100
Epoch 00077: val_loss did not improve
 - 1s - loss: 0.3453 - val_loss: 0.3426
Epoch 78/100
Epoch 00078: val_loss did not improve
 - 1s - loss: 0.3390 - val_loss: 0.3340
E

In [19]:
multiple_places(place_groups_junctions, 'place_groups_junctions')

Train on 34780 samples, validate on 8696 samples
Epoch 1/100
Epoch 00001: val_loss improved from inf to 12.48655, saving model to simple_lstm_flow_ChIJsVzlKx5TUkYRjzAT9WOeWcA.best.hdf5
 - 4s - loss: 15.5054 - val_loss: 12.4866
Epoch 2/100
Epoch 00002: val_loss improved from 12.48655 to 6.72819, saving model to simple_lstm_flow_ChIJsVzlKx5TUkYRjzAT9WOeWcA.best.hdf5
 - 1s - loss: 8.2119 - val_loss: 6.7282
Epoch 3/100
Epoch 00003: val_loss improved from 6.72819 to 3.26741, saving model to simple_lstm_flow_ChIJsVzlKx5TUkYRjzAT9WOeWcA.best.hdf5
 - 1s - loss: 4.4471 - val_loss: 3.2674
Epoch 4/100
Epoch 00004: val_loss improved from 3.26741 to 1.90288, saving model to simple_lstm_flow_ChIJsVzlKx5TUkYRjzAT9WOeWcA.best.hdf5
 - 1s - loss: 2.4458 - val_loss: 1.9029
Epoch 5/100
Epoch 00005: val_loss improved from 1.90288 to 1.73992, saving model to simple_lstm_flow_ChIJsVzlKx5TUkYRjzAT9WOeWcA.best.hdf5
 - 1s - loss: 1.7980 - val_loss: 1.7399
Epoch 6/100
Epoch 00006: val_loss improved from 1.73992 

Epoch 00067: val_loss did not improve
 - 1s - loss: 0.7895 - val_loss: 0.7273
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 1s - loss: 0.7813 - val_loss: 0.7291
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 1s - loss: 0.7882 - val_loss: 0.7374
Epoch 70/100
Epoch 00070: val_loss did not improve
 - 1s - loss: 0.7823 - val_loss: 0.7450
Epoch 71/100
Epoch 00071: val_loss did not improve
 - 1s - loss: 0.7827 - val_loss: 0.7743
Epoch 72/100
Epoch 00072: val_loss did not improve
 - 1s - loss: 0.7812 - val_loss: 0.8374
Epoch 73/100
Epoch 00073: val_loss improved from 0.72241 to 0.71333, saving model to simple_lstm_flow_ChIJsVzlKx5TUkYRjzAT9WOeWcA.best.hdf5
 - 1s - loss: 0.7802 - val_loss: 0.7133
Epoch 74/100
Epoch 00074: val_loss did not improve
 - 2s - loss: 0.7823 - val_loss: 0.7323
Epoch 75/100
Epoch 00075: val_loss did not improve
 - 2s - loss: 0.7763 - val_loss: 0.7573
Epoch 76/100
Epoch 00076: val_loss did not improve
 - 1s - loss: 0.7809 - val_loss: 0.7280
Epoch 77/100
E